# Visualizing sector allocations of a mutual fund through time

A version of this notebook using normalized sector allocations can be found [here](https://github.com/stockviz/plutons/blob/master/mutual-fund-sector-allocations.R.ipynb)

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

mfi <- MutualFundsIndia()

In [ ]:
#Franklin India Prima Fund-Growth (https://github.com/stockviz/plutons/blob/master/mutual-fund-drawdown.R.ipynb)
scode <- 112932 
sname <- "Mirae Asset Emerging Bluechip Fund"

sectorAlloc <- mfi$Portfolio() %>%
    filter(SCHEME_CODE == scode 
           & INSTRUMENT == 'EQUITY' 
           & !is.null(WEIGHTAGE)
           & WEIGHTAGE > 0
           & (INSTRUMENT_TYPE == 'STOCK' | INSTRUMENT_TYPE == 'E' | INSTRUMENT_TYPE == '' | is.null(INSTRUMENT_TYPE))) %>%
    group_by(PORTFOLIO_DATE, INDUSTRY_BSE) %>%
    summarize(ALLOC = sum(WEIGHTAGE)) %>%
    arrange(PORTFOLIO_DATE, INDUSTRY_BSE) %>%
    collect()

saDf <- sectorAlloc %>%
    mutate(SECTOR = replace(INDUSTRY_BSE, INDUSTRY_BSE == '-' | is.na(INDUSTRY_BSE), 'UNK')) %>%
    as.data.frame()

saDf$SCALE <- scale(saDf$ALLOC)

In [ ]:
firstDate <- min(saDf$PORTFOLIO_DATE)
lastDate <- max(saDf$PORTFOLIO_DATE)
xAxisTicks <- seq(from=firstDate, to=lastDate, length.out=10)
                  
ggplot(saDf, aes(x = PORTFOLIO_DATE, y = SECTOR, fill = SCALE)) +
    theme_economist() +
    theme(axis.text.x = element_text(angle = 90, hjust = 1)) +
    geom_tile() +
    scale_x_date(breaks = xAxisTicks) +
    #geom_text(aes(label= sprintf("%.2f", ALLOC)), hjust = 1) +
    scale_fill_gradientn(colors=c("green", "red")) +
    guides(fill=F) +
    labs(x = "", y="", fill="", color="", title=sprintf("%s Sector Weights", sname)) 

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)